## Finding Association Rules on Grocery Dataset
In this Notebook, we will be implementing an algorithm for mining association rules from a dataset. We will test our algorithm with a small synthetic (artificial) dataset, before we use the algorithm to find association rules from a larger dataset - the [grocery dataset](https://www.kaggle.com/irfanasrullah/groceries).

Our Notebooks in CSMODEL are designed to be guided learning activities. To use them, simply through the cells from top to bottom, following the directions along the way. If you find any unclear parts or mistakes in the Notebooks, email me at thomas.tiam-lee@dlsu.edu.ph.

## Import
Import **pandas** and **matplotlib**.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Synthetic Dataset
Before we use a more complicated dataset, we will first test our algorithm using a synthetic (artificial) dataset created using random numbers. The dataset contains 20 distinct items. There are 300 observations representing the baskets in the market-basket model. Each observation (basket) contains at most 8 items.

Let's first create the synthetic dataset using the [`choice`](https://docs.scipy.org/doc//numpy-1.10.4/reference/generated/numpy.random.choice.html) function of `numpy`. You may check the documentation of the function for further information. We have set the same seed to have the same values in the synthetic dataset.

In [2]:
np.random.seed(1)
baskets = [np.sort(np.random.choice(20, size=(np.random.randint(1, 9)), replace=False)) for i in range(300)]

Let's display the contents of the synthetic dataset. It should list 300 baskets with its contents.

In [3]:
for i, basket in enumerate(baskets):
    print('Basket', i, basket)

Basket 0 [ 3 10 14 15 17 18]
Basket 1 [ 2  3  5  8 17]
Basket 2 [ 2 11 12 16 17 18]
Basket 3 [16]
Basket 4 [ 4  5 12 16]
Basket 5 [ 0  1  3  9 17 18]
Basket 6 [11]
Basket 7 [ 3  8 13]
Basket 8 [ 2  5 12 13 19]
Basket 9 [ 1  9 12 14 19]
Basket 10 [6 7]
Basket 11 [ 1  6  8  9 13 15 17 18]
Basket 12 [ 3  6  9 11 12 14 18 19]
Basket 13 [ 1  3  4  5  7  8 11]
Basket 14 [ 1  2  9 11 15 16 18 19]
Basket 15 [ 3  4  8  9 10 14]
Basket 16 [ 8  9 12 13 18]
Basket 17 [1 5 8]
Basket 18 [6 7]
Basket 19 [ 3 19]
Basket 20 [ 5  8 13 15]
Basket 21 [ 1  3  4 15 19]
Basket 22 [ 0  3 12]
Basket 23 [ 1  5  7  9 14 17]
Basket 24 [10 14 17]
Basket 25 [0 3 4]
Basket 26 [ 4  7 10 18]
Basket 27 [ 2  5 11 17 18]
Basket 28 [ 1  3  7  8 11 17 18]
Basket 29 [0 2 4 9]
Basket 30 [ 3 15 17 19]
Basket 31 [8]
Basket 32 [ 3  9 11]
Basket 33 [ 4  5  7  8  9 16 17]
Basket 34 [ 2  6  7 19]
Basket 35 [5]
Basket 36 [ 1  3  7 10 12 15 16]
Basket 37 [ 1 14]
Basket 38 [ 2  3  6 13 18 19]
Basket 39 [ 2  6 11 14 16]
Basket 40 [3]
B

As of now, our dataset is represented as a list of list. Instead of using this representation, we will convert our dataset to a matrix represented as a `pandas` `DataFrame`. The `DataFrame` will contain 300 rows - equivalent to the number of observations in the dataset, and 20 columns - equivalent to the number of distinct items in the dataset. The value in the cell in row `x` and column `y` is 1 if item `y` is in observation (basket) `x`, otherwise, the value in the cell in row `x` and column `y` is 0.

In [4]:
syn_df = pd.DataFrame([[0 for _ in range(20)] for _ in range(300)], columns=[i for i in range(20)])

for i, basket in enumerate(baskets):
    syn_df.iloc[i, basket] = 1

Let's check the `DataFrame` representing the synthetic dataset here. In row `0`, the `DataFrame` should contain the value `1` in columns `3`, `10`, `14`, `15`, `17`, and `18`. All other columns in row `0` should contain the value 0. You may check the other values based on the list-of-list representation that we have displayed earlier.

In [5]:
print(syn_df)

     0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15  16  17  \
0     0   0   0   1   0   0   0   0   0   0   1   0   0   0   1   1   0   1   
1     0   0   1   1   0   1   0   0   1   0   0   0   0   0   0   0   0   1   
2     0   0   1   0   0   0   0   0   0   0   0   1   1   0   0   0   1   1   
3     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0   
4     0   0   0   0   1   1   0   0   0   0   0   0   1   0   0   0   1   0   
..   ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..   
295   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0   0   
296   0   1   0   0   0   1   0   0   0   0   1   1   0   1   0   0   0   1   
297   0   1   0   0   0   0   0   0   0   0   0   0   0   0   1   0   0   0   
298   1   0   0   1   0   0   0   0   0   0   0   1   0   0   0   0   0   1   
299   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   

     18  19  
0     1   0  
1     0   0  
2     1  

## Rule Miner
Open `rule_miner.py` file. Some of the functions in the `RuleMiner` class are not yet implemented. We will implement the missing parts of this class.

Import the `RuleMiner` class

In [6]:
from rule_miner import RuleMiner

Instantiate a `RuleMiner` object with `support_t` equal to `10` and `confidence_t` equal to `0.6`. The field `support_t` represents the support threshold, while the field `confidence_t` represents the confidence threshold.

In [7]:
rule_miner = RuleMiner(10, 0.6)

Open `rule_miner.py` file and complete the `get_support()` function. This function returns the support for an itemset. The support of an itemset refers to the number of baskets wherein the itemset is present.

Implement the `get_support()` function. Inline comments should help you in completing the contents of the function. Upon implementing the function, execute the code below then answer the questions.

In [8]:
print(rule_miner.get_support(syn_df, [0]))
print(rule_miner.get_support(syn_df, [0, 1]))
print(rule_miner.get_support(syn_df, [0, 1, 2]))

64
16
4


**Question:** What is the support of the itemset `{0}`? 
- 64

**Question:** What is the support of the itemset `{0, 1}`? 
- 16

**Question:** What is the support of the itemset `{0, 1, 2}`? 
- 4

Open `rule_miner.py` file again and complete the `get_frequent_itemsets()` function. This function returns a list frequent itemsets in the dataset. The support of each frequent itemset should be greater than or equal to the support threshold.

Implement the `get_frequent_itemsets()` function. Inline comments should help you in completing the contents of the function. Upon implementing the function, execute the code below then answer the questions.

In [9]:
frequent_itemsets = rule_miner.get_frequent_itemsets(syn_df)
print(frequent_itemsets)

[[1, 12, 7], [16, 1, 12], [1, 19, 12], [8, 17, 2]]


**Question:** List all the frequent itemsets in the dataset, given the support threshold `10`.
- [[1, 12, 7], [16, 1, 12], [1, 19, 12], [8, 17, 2]]

Using the `get_rules()` function in `rule_miner.py`, let us list all the possible rules for all frequent itemsets in our dataset. The `get_rules()` function returns a list of rules produced from an itemset.

In [10]:
for itemset in frequent_itemsets:
    print(rule_miner.get_rules(itemset))

[[[1, 12], [7]], [[7], [1, 12]], [[1, 7], [12]], [[12], [1, 7]], [[12, 7], [1]], [[1], [12, 7]]]
[[[16, 1], [12]], [[12], [16, 1]], [[16, 12], [1]], [[1], [16, 12]], [[1, 12], [16]], [[16], [1, 12]]]
[[[1, 19], [12]], [[12], [1, 19]], [[1, 12], [19]], [[19], [1, 12]], [[19, 12], [1]], [[1], [19, 12]]]
[[[8, 17], [2]], [[2], [8, 17]], [[8, 2], [17]], [[17], [8, 2]], [[17, 2], [8]], [[8], [17, 2]]]


Upon getting all the possible rules based on our most frequent itemsets, we should check if the confidence of each rule is greater than or equal to the confidence threshold that we set.

To do this, open `rule_miner.py` file again and complete the `get_confidence()` function. This function returns the confidence for a rule. Suppose that we want to find the rule is `{1, 2} -> {3}`, then the confidence for the rule is the support of `{1, 2, 3}` divided by the support of `{1, 2}`. In this code, we represent a rule using a list which contains 2 lists -  the first list contains the left-hand side of the rule (which in our example is `{1, 2}`), and the second list contains the right-hand side of the rule (which in our example is `{3}`).

Implement the `get_confidence()` function. Inline comments should help you in completing the contents of the function. Upon implementing the function, execute the code below then answer the questions.

In [11]:
print(rule_miner.get_confidence(syn_df, [[1, 2], [3]]))
print(rule_miner.get_confidence(syn_df, [[4, 5], [6]]))
print(rule_miner.get_confidence(syn_df, [[7, 8], [9]]))

0.21428571428571427
0.06666666666666667
0.09090909090909091


**Question:** What is the confidence of the rule `{1, 2} -> {3}`? 
- 0.21428571428571427

**Question:** What is the confidence of the rule `{4, 5} -> {6}`? 
- 0.06666666666666667

**Question:** What is the confidence of the rule `{7, 8} -> {9}`? 
- 0.09090909090909091

We have now completed all functions necessary for our rule miner. The only function left to implement is the `get_association_rules()` function, which integrates all of these functions together.

Open `rule_miner.py` file again and complete the `get_association_rules()` function. This function returns a list of association rules with support greater than or equal to the support threshold `support_t` and confidence greater than or equal to the confidence threshold `confidence_t`.

Implement the `get_association_rules()` function. Inline comments should help you in completing the contents of the function. Upon implementing the function, execute the code below then answer the questions.

With `support_t` equal to `10`, and `confidence_t` equal to `0.6`, let's get the association rules from this dataset.

In [12]:
rules = rule_miner.get_association_rules(syn_df)
print(rules)

[[[12, 7], [1]]]


**Question:** What is/are the association rules that we derived from the dataset?
- When a basket has items 12 and 7, they would most likely have item 1 as well.

## Grocery Dataset
For this notebook, we will work on a dataset called `grocery dataset`. This dataset contains 9835 rows which represents transactions by customers shopping for groceries. The dataset contains 169 unique items.

The dataset is provided to you as a `.csv` file. `.csv` means comma-separated values. You can open the file in Notepad to see how it is exactly formatted.

If you view the `.csv` file in Excel, you can see that our dataset contains a list of items bought by a customer for each single transaction, represented in rows.

In [13]:
temp_df = pd.read_csv("groceries.csv", header=None)

Let's convert the items, represented as strings, to integers. To do this, let's create a dictionary that will contain the mapping for each item string to its corresponding integer. The dictionary should contain 169 unique strings, with integer mapping from 0 to 168.

In [14]:
values = temp_df.values.ravel()
values = [value for value in pd.unique(values) if not pd.isnull(value)]

value_dict = {}
for i, value in enumerate(values):
    value_dict[value] = i
    
print(value_dict)

{'citrus fruit': 0, 'semi-finished bread': 1, 'margarine': 2, 'ready soups': 3, 'tropical fruit': 4, 'yogurt': 5, 'coffee': 6, 'whole milk': 7, 'pip fruit': 8, 'cream cheese': 9, 'meat spreads': 10, 'other vegetables': 11, 'condensed milk': 12, 'long life bakery product': 13, 'butter': 14, 'rice': 15, 'abrasive cleaner': 16, 'rolls/buns': 17, 'UHT-milk': 18, 'bottled beer': 19, 'liquor (appetizer)': 20, 'potted plants': 21, 'cereals': 22, 'white bread': 23, 'bottled water': 24, 'chocolate': 25, 'curd': 26, 'flour': 27, 'dishes': 28, 'beef': 29, 'frankfurter': 30, 'soda': 31, 'chicken': 32, 'sugar': 33, 'fruit/vegetable juice': 34, 'newspapers': 35, 'packaged fruit/vegetables': 36, 'specialty bar': 37, 'butter milk': 38, 'pastry': 39, 'processed cheese': 40, 'detergent': 41, 'root vegetables': 42, 'frozen dessert': 43, 'sweet spreads': 44, 'salty snack': 45, 'waffles': 46, 'candy': 47, 'bathroom cleaner': 48, 'canned beer': 49, 'sausage': 50, 'brown bread': 51, 'shopping bags': 52, 'bev

As of now, the `DataFrame` representation of the transaction contains 9835 rows, wherein each row contains a list of string representing the items bought for each transaction. We want to convert this representation to a list of list, with the corresponding integers as value instead of the strings.

In [15]:
temp_df = temp_df.stack().map(value_dict).unstack()

baskets = []
for i in range(temp_df.shape[0]):
    basket = np.sort([int(x) for x in temp_df.iloc[i].values.tolist() if str(x) != 'nan'])
    baskets.append(basket)

Let's display the contents of the dataset. It should list 9835 baskets with its contents.

In [16]:
for i, basket in enumerate(baskets):
    print('Basket', i, basket)

Basket 0 [0 1 2 3]
Basket 1 [4 5 6]
Basket 2 [7]
Basket 3 [ 5  8  9 10]
Basket 4 [ 7 11 12 13]
Basket 5 [ 5  7 14 15 16]
Basket 6 [17]
Basket 7 [11 17 18 19 20]
Basket 8 [21]
Basket 9 [ 7 22]
Basket 10 [ 4 11 23 24 25]
Basket 11 [ 0  4  5  7 14 24 26 27 28]
Basket 12 [29]
Basket 13 [17 30 31]
Basket 14 [ 4 32]
Basket 15 [14 33 34 35]
Basket 16 [34]
Basket 17 [36]
Basket 18 [25]
Basket 19 [37]
Basket 20 [11]
Basket 21 [38 39]
Basket 22 [7]
Basket 23 [ 4  9 35 40 41]
Basket 24 [ 4 11 17 27 42 43 44 45 46 47 48]
Basket 25 [24 49]
Basket 26 [5]
Basket 27 [17 25 31 50]
Basket 28 [11]
Basket 29 [31 34 35 49 51 52]
Basket 30 [ 5 24 37 53]
Basket 31 [11 17 24 54 55 56 57]
Basket 32 [ 7 11 33 42 53]
Basket 33 [ 7 11 16 31 58 59 60 61]
Basket 34 [29 41 62]
Basket 35 [31 39]
Basket 36 [34]
Basket 37 [49]
Basket 38 [ 7 11 42 63]
Basket 39 [ 0 35 64]
Basket 40 [17 31 37 49 50 52]
Basket 41 [ 4  5  6  7 22 31 33 39 42 46 47 51 65]
Basket 42 [ 5 59]
Basket 43 [49]
Basket 44 [ 5  9 17 24 31 35 38 66]


Basket 495 [ 19 146]
Basket 496 [ 19 130]
Basket 497 [ 0 11 31]
Basket 498 [11 56 58 78]
Basket 499 [11 52]
Basket 500 [41 94]
Basket 501 [ 7 24 63]
Basket 502 [ 8 52 94]
Basket 503 [19 31 33]
Basket 504 [ 4  6  8 17 34 47]
Basket 505 [ 9 19 23 24 31 34 42 43 66 67]
Basket 506 [31 67]
Basket 507 [ 7 26]
Basket 508 [  4   5  11  24 102]
Basket 509 [23 51]
Basket 510 [33 51 56]
Basket 511 [  4   6  24  42  47  52  82 126]
Basket 512 [65]
Basket 513 [ 6 41 56 96]
Basket 514 [31]
Basket 515 [7]
Basket 516 [19]
Basket 517 [  6   7  25  29  31  35  36  57  58  59  69  74  83 117 121 125 140]
Basket 518 [ 5 11]
Basket 519 [ 5 24 28]
Basket 520 [88]
Basket 521 [24 47]
Basket 522 [ 0 11 33 34 38 56 59]
Basket 523 [ 7 11 17 42 54]
Basket 524 [36 80]
Basket 525 [ 2 11]
Basket 526 [31]
Basket 527 [ 2  5  7 24 34 45 83]
Basket 528 [ 5 93]
Basket 529 [ 0 17 31 66]
Basket 530 [67]
Basket 531 [120]
Basket 532 [31]
Basket 533 [ 4  7 23 52 53 98]
Basket 534 [ 31 120]
Basket 535 [23 24 31 46]
Basket 536 

Basket 836 [5]
Basket 837 [5]
Basket 838 [24]
Basket 839 [ 7 59 60 65]
Basket 840 [ 7 17 36 39]
Basket 841 [30 31]
Basket 842 [  1   7  21  31  52 151 153]
Basket 843 [ 0  6  7 11 14 27 28 31 33 36 60 88]
Basket 844 [ 0 17 26 29 34 42 68 92]
Basket 845 [ 5  6  7 49 57 78]
Basket 846 [11 60]
Basket 847 [11 29]
Basket 848 [  5   6   7  11  59  63  66  82  94 101]
Basket 849 [  7  11  26  33  35  65  78 138]
Basket 850 [ 4  5  7 14 42 57 59 60 68 85 97]
Basket 851 [6]
Basket 852 [  1   3  11  17  24  35  50  56  58  61 117]
Basket 853 [  9  28  42  46  53  60 123]
Basket 854 [  2   4   8  11  13  14  16  17  32  33  36  42 132]
Basket 855 [  2   7  34  52  59  60 145]
Basket 856 [67 90]
Basket 857 [60]
Basket 858 [19 25 29 42 52]
Basket 859 [ 7 11 17 18 19 24 31 32 34 58 60 65]
Basket 860 [  7   8  11  31  32  34  42  51  52  54  58  66  83 104 153]
Basket 861 [ 4  5  7 11 17 19 24 29 42 45 51 52 68 93]
Basket 862 [36 73]
Basket 863 [33]
Basket 864 [ 5  7 17 29 30 31 36 42 50 63 77 97]
Ba

Basket 1161 [ 0  4  8 13]
Basket 1162 [31]
Basket 1163 [75]
Basket 1164 [19 31 42 97]
Basket 1165 [11 58]
Basket 1166 [ 7 18 34 71]
Basket 1167 [ 4 11 31 47 52]
Basket 1168 [  2   6  11  49  50  52  57  74  98 142 149]
Basket 1169 [ 11  20  24  49  50  51  52  79 123]
Basket 1170 [  4   5   6  11  15  24  27  31  34  45  49  58  63 139]
Basket 1171 [  0   8  50  51  52 140]
Basket 1172 [ 25  62 149]
Basket 1173 [ 39  45  52  62 149]
Basket 1174 [  5  11  24  40 102 150]
Basket 1175 [21]
Basket 1176 [21 24]
Basket 1177 [ 21  56 116]
Basket 1178 [ 18  31  57 121]
Basket 1179 [24 31 47 75 85]
Basket 1180 [17 49 79]
Basket 1181 [85]
Basket 1182 [ 5  9 11 19 23 25 29 34 38 60]
Basket 1183 [ 4 24 98]
Basket 1184 [ 5 11 14 17 27 29 42]
Basket 1185 [17 94]
Basket 1186 [ 19  42  59  60 139]
Basket 1187 [ 7 17 50 54 56 73 74 99]
Basket 1188 [ 7 26 29 32 58 60]
Basket 1189 [ 0  7  9 17 34 39 40 50 75 78]
Basket 1190 [52]
Basket 1191 [49]
Basket 1192 [31 49]
Basket 1193 [ 5 17 18]
Basket 1194 [17 

Basket 1494 [ 0  5  7 17 42 50 54]
Basket 1495 [ 6 12 17 50]
Basket 1496 [ 2 11 29 86]
Basket 1497 [45 49 52]
Basket 1498 [49]
Basket 1499 [17 31 46 52]
Basket 1500 [ 7 24 35 53]
Basket 1501 [21 42 51]
Basket 1502 [5]
Basket 1503 [52 92]
Basket 1504 [ 0  4  6 15 17 54]
Basket 1505 [28 57]
Basket 1506 [11 18 24 38 51]
Basket 1507 [ 7 11 17 18 26 32 35]
Basket 1508 [  2   7  31  41  83 123]
Basket 1509 [17]
Basket 1510 [  5  14  17  31 151]
Basket 1511 [ 11  50  51  52 101]
Basket 1512 [17]
Basket 1513 [11 17]
Basket 1514 [ 7 30]
Basket 1515 [35]
Basket 1516 [  0   4  11  17  34  39  42  51  57  62  69  71  73  92 151]
Basket 1517 [ 14  35  57  74  92 139]
Basket 1518 [  7  26  35  77  79 104]
Basket 1519 [  5  24  35  44  78 127]
Basket 1520 [60 87]
Basket 1521 [51 78]
Basket 1522 [  0   7  41  65  67 103 115]
Basket 1523 [ 5 74 92]
Basket 1524 [17 31 38 50]
Basket 1525 [11 14 28 39 57]
Basket 1526 [30 33 51]
Basket 1527 [17 65 94]
Basket 1528 [17 31 79]
Basket 1529 [19]
Basket 1530 [46

...

Basket 8387 [ 7 11 31 39 51 58 67]
Basket 8388 [19 24 30 67 88]
Basket 8389 [52 75]
Basket 8390 [ 7 11 32 39]
Basket 8391 [78]
Basket 8392 [17]
Basket 8393 [  5  34  56 124]
Basket 8394 [ 4  5  7  8 11 14 30 50 51 62 82 90]
Basket 8395 [31 50]
Basket 8396 [ 0 18 24 34 39 51 92]
Basket 8397 [ 1  5  7 11 25 31 34 35 56 69 90]
Basket 8398 [ 7 53]
Basket 8399 [17 53 75]
Basket 8400 [104]
Basket 8401 [11 54]
Basket 8402 [ 17  50 122]
Basket 8403 [  4   5  17  31  33  41  62  77  78 121]
Basket 8404 [ 6  7 25 38 47 68]
Basket 8405 [39 74]
Basket 8406 [  7   9  23  25  26  33  60  69  72  74 165]
Basket 8407 [ 57  76  98 109 134]
Basket 8408 [  5   7  31  69  79  90  98 130 134]
Basket 8409 [24 31 40]
Basket 8410 [51]
Basket 8411 [45 49 82 90]
Basket 8412 [ 0  4  5 11 14 26 27 34 36 38 46 47 50 51 52 62]
Basket 8413 [ 4 17 35 47 90]
Basket 8414 [11 21 29 46]
Basket 8415 [ 19 129]
Basket 8416 [65]
Basket 8417 [88]
Basket 8418 [31 45 67 75]
Basket 8419 [17 31 50]
Basket 8420 [25]
Basket 8421 [ 

Basket 8719 [ 4 31 65 67 68 75]
Basket 8720 [ 0 11 32]
Basket 8721 [24]
Basket 8722 [ 13  21  40  46  52  59  62  68  70 110 120]
Basket 8723 [37 49]
Basket 8724 [28 62 94]
Basket 8725 [ 11  14  31  54  60  93 119 121]
Basket 8726 [ 5 26]
Basket 8727 [17 51]
Basket 8728 [46]
Basket 8729 [11 29 35]
Basket 8730 [29]
Basket 8731 [14 19]
Basket 8732 [21 31 34 50 57 65 78]
Basket 8733 [  0   5   8  30  34  50  51  65  74  92 143]
Basket 8734 [25 47]
Basket 8735 [18 47 57]
Basket 8736 [  4  17  31  50  52 100]
Basket 8737 [9]
Basket 8738 [19 52]
Basket 8739 [17 34 35 85]
Basket 8740 [17]
Basket 8741 [  7  17  25  35  45  47 109 113]
Basket 8742 [  5   7   8   9  11  23  26  29  38  39  42  47  50  62  90  97 112]
Basket 8743 [  4  17  45  60  78 121]
Basket 8744 [  7  14  24  26  29  62  69  77 121]
Basket 8745 [17 25 47 50 52]
Basket 8746 [ 1  4 70 76 77]
Basket 8747 [11 42 62 65]
Basket 8748 [7]
Basket 8749 [ 2  5 29]
Basket 8750 [ 34  79 118]
Basket 8751 [ 7 33 35 39]
Basket 8752 [  4   7

Basket 9046 [ 2 11 34 35 51 63 89]
Basket 9047 [24 26 31]
Basket 9048 [ 4  7 11 40 42]
Basket 9049 [17]
Basket 9050 [ 18  34  63 123]
Basket 9051 [31]
Basket 9052 [96]
Basket 9053 [ 24  52  67 130]
Basket 9054 [  5   7  11  60  67  72  88 121]
Basket 9055 [ 0 11 13 86 94]
Basket 9056 [ 0 17 24 53 65 75 76]
Basket 9057 [17 96]
Basket 9058 [49]
Basket 9059 [49]
Basket 9060 [33]
Basket 9061 [  4  58  59  72  79  80 123]
Basket 9062 [ 5  6  7 11 29 30 38 68 96]
Basket 9063 [28]
Basket 9064 [ 7 11 17 58]
Basket 9065 [  0   7  11  17  19  30  35  42  62  92 131]
Basket 9066 [ 7 11 19 39 60]
Basket 9067 [35]
Basket 9068 [31 42 59 60]
Basket 9069 [ 2  7 17 23 29 35]
Basket 9070 [ 7 13 17 24 60 73]
Basket 9071 [  0   2   7  17  60 100]
Basket 9072 [ 6  7 11 12 19 23 31 33 34 52]
Basket 9073 [ 7 27 31 33 42 67]
Basket 9074 [ 11  30  39  54 100 121 147]
Basket 9075 [11]
Basket 9076 [  4   5  13  24  26  50  52  56 115]
Basket 9077 [ 5 16]
Basket 9078 [17 49 50]
Basket 9079 [ 2  5  7 38 54 72]
Bas

Basket 9356 [49]
Basket 9357 [88]
Basket 9358 [17 19 31 45 50 57 63 65 82]
Basket 9359 [41 87]
Basket 9360 [ 14  24  28  37  52 129]
Basket 9361 [ 19  75 129]
Basket 9362 [ 7 26 57 70]
Basket 9363 [56]
Basket 9364 [ 4 26 35]
Basket 9365 [ 4 11 14 65 68]
Basket 9366 [ 11 123]
Basket 9367 [ 8 32 55 94]
Basket 9368 [130]
Basket 9369 [25 53]
Basket 9370 [  7  11  17  50  52 109 131]
Basket 9371 [5]
Basket 9372 [ 7 35]
Basket 9373 [74]
Basket 9374 [ 0  5  7  9 14 50 51 54 58 60 68 70 90 97]
Basket 9375 [ 6 19]
Basket 9376 [  7  26  90 143]
Basket 9377 [  1  45  50  57 108 117]
Basket 9378 [  7  17  30  50  74 114 147]
Basket 9379 [60]
Basket 9380 [78]
Basket 9381 [60]
Basket 9382 [ 30 149]
Basket 9383 [ 7 14 24 45 65]
Basket 9384 [ 5 78]
Basket 9385 [57]
Basket 9386 [ 2  5 31 42 63 93]
Basket 9387 [24]
Basket 9388 [11 18]
Basket 9389 [24]
Basket 9390 [ 7  8 27 36 42 65 89]
Basket 9391 [57]
Basket 9392 [ 4 17 42]
Basket 9393 [ 5 13 31 57 77 85]
Basket 9394 [18 31 50 63 94]
Basket 9395 [  5  

Basket 9656 [  4   7   9  11  17  24  28  29  82 125]
Basket 9657 [ 1 80]
Basket 9658 [  4   7  10  30  34  51  52  72 125]
Basket 9659 [17 19]
Basket 9660 [  6   7  11  14  17  24  30  50  54 123]
Basket 9661 [ 11  17  35  42  54 130 139]
Basket 9662 [ 11  31  51  52  65 129]
Basket 9663 [12 43]
Basket 9664 [7]
Basket 9665 [11]
Basket 9666 [38 96]
Basket 9667 [19]
Basket 9668 [ 5  7 29 39]
Basket 9669 [ 26  51  52 103]
Basket 9670 [19]
Basket 9671 [39]
Basket 9672 [94]
Basket 9673 [ 7 11 26 37 41 46 57]
Basket 9674 [ 7 13 14 17 26 40 51 67]
Basket 9675 [  4   7  25  38  39  51  63  70  74 109]
Basket 9676 [  4  39  46  63 119 161]
Basket 9677 [ 5  7  8 26 38 50 70 97]
Basket 9678 [11 65]
Basket 9679 [ 50  68  70 134]
Basket 9680 [  6  27  31  47  51  56  65  69  74 147]
Basket 9681 [11]
Basket 9682 [ 27  31  33  39  42  50  63 109]
Basket 9683 [ 4  5  6 14 31 34 37 59 92]
Basket 9684 [46 52]
Basket 9685 [ 39 126]
Basket 9686 [19 40 58]
Basket 9687 [41 46]
Basket 9688 [ 6  8 11]
Basket

As of now, our dataset is represented as a list of list. Instead of using this representation, we will convert our dataset to a matrix represented as a `pandas` `DataFrame`. The `DataFrame` will contain 9835 rows - equivalent to the number of observations in the dataset, and 169 columns - equivalent to the number of distinct items in the dataset. The value in the cell in row `x` and column `y` is 1 if item `y` is in observation (basket) `x`, otherwise, the value in the cell in row `x` and column `y` is 0.

In [17]:
grocery_df = pd.DataFrame([[0 for _ in range(169)] for _ in range(9835)], columns=values)

for i, basket in enumerate(baskets):
    grocery_df.iloc[i, basket] = 1

Let's check the `DataFrame` representing the dataset here. In row `0`, the `DataFrame` should contain the value `1` in columns `citrus fruit`, `semi-finished bread`, `margarine`, and `ready soups`. All other columns in row `0` should contain the value 0. You may check the other values based on the list-of-list representation that we have displayed earlier.

In [18]:
print(grocery_df)

      citrus fruit  semi-finished bread  margarine  ready soups  \
0                1                    1          1            1   
1                0                    0          0            0   
2                0                    0          0            0   
3                0                    0          0            0   
4                0                    0          0            0   
...            ...                  ...        ...          ...   
9830             1                    0          0            0   
9831             0                    0          0            0   
9832             1                    0          0            0   
9833             0                    1          0            0   
9834             0                    0          0            0   

      tropical fruit  yogurt  coffee  whole milk  pip fruit  cream cheese  \
0                  0       0       0           0          0             0   
1                  1       1       1     

Instantiate a `RuleMiner` object with `support_t` equal to `85` and `confidence_t` equal to `0.6`. The field `support_t` represents the support threshold, while the field `confidence_t` represents the confidence threshold.

In [19]:
rule_miner = RuleMiner(85, 0.6)

With `support_t` equal to `85`, and `confidence_t` equal to `0.6`, let's get the association rules from this dataset.

In [20]:
rules = rule_miner.get_association_rules(grocery_df)
print(rules)

[[['butter', 'yogurt'], ['whole milk']]]


**Question:** What is/are the association rules that we derived from the dataset?
- When the the items yogurt and butter are selected, there's a high likelihood that a whole milk is also bought.